<a href="https://colab.research.google.com/github/DLesmes/GPC/blob/main/Gemma_Python_Chat_(GPC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'data-assistants-with-gemma:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F64148%2F7669720%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240325%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240325T231817Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D32aa0a51ec3e9a672f8b869d73a28c01e66c38624090ea36969400aa62d181fc6b2e01e74dc198afe778f6729ff2b7cb7dd072a0c5c0f67ba228cabb8f9e8ef5cff7040487dab6cb955bd4756c50b7453afab1a634030cee004148dd342fc04318640d2d671b2fef11f7825e3cfcbf63ff07501ecc5298355bd07c1b94a9cd9f034b67fc1f4f7abdff2ec05b2e7fc892cdc642249e73c17d8c9ff4cb49823665f023dc21c25f4e11d0b20a3cf57469420560d82ca5e6b6d2c1da7114cdb181c011b50baac717564b673f771f4065a580f2f476404c917261c8aec7599226f3ce32e5656096eb054bd6a7262cc96a70c0a628d4ce459a8aa50e52c6b8a9255a20,pythonquestions:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F262%2F726715%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240325%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240325T231817Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D458d762ee45017b47c9fc3e5e0e8399ba4c4c7c81ccd05e73da9c8a5cba7cb687cf7320a1dfb30f4d792231134af8c8cf950f1290dd7d0c301e7feed0b5162b27e9b94f60e52a43eb0b4dcc363d58ea5d1165721b0087b9ed11ddb8352c1b0b3e9d0bf0a7d2ba23017fa56c58ffe9cef0586e7b9a59e13dd5a0ea8565491c43ca0a358c45e11fceba766ea7c54750cabc997b51b531b7533ede1d52b62bec2b97b48574256a8c1792e8757c5870689b9a62bb9c4194e57586b674bf0637e034708ea4e371946707d5590a7fa6a53a4a8f4fa64851584c77a60d21b0f7cfa546ed0f5636756900ddd8259c711e8afef6d84bbf6e2b6ac8c25b87cd8d11cb9bf12,gemma/transformers/2b/2:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F6216%2F11384%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240325%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240325T231817Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D561bb01165eb565da289a57e7d98500b1b7211c7d4cc5cff932d7464174a0406895f1d411ab68619ca573c5694659c70e8d4d34fc6cd8ec19d086138bdc9cb32ed9b5b24fa8edbe19b3bf546e3045011929466254ca675485807c94c4e514ef9efc993e4926c68de6e624f03172176a6dfb7267a9ec7550b5b34bab0aa90b01e4da46fb42b0f5c8f56be5d68665946acd1cd7f32265b534db20a631ffb9e2314aa2d3a763a9cf5b270af5ece11aa1d2a4f2c64ba15cef6a5e59e67b11887ec69982dd6f5f947774f97c34fc1bf5521ea8b0332afb5745063d3b8734463ea98984cf57feb47eb7776fac40cd0192fb470a9f3bdabdf09f8e3e6209c2db9f26212'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


<a href="https://colab.research.google.com/github/DLesmes/GPC/blob/main/GPC_(Gemma_Python_Chat).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GCP 🤖 Gemma Python Chatbot

![](https://i.ibb.co/8xZNc32/Gemma.png)

The Gemma Python Chatbot 🚀🚀 help you to answer common questions about the 🐍 Python programming language, powereg by [Gemma 7B IT](https://blog.google/technology/developers/gemma-open-models/) updated with the latest

## Requirements

In [ ]:
#base
import os
import json
import re
import csv
import pandas as pd
from typing import *
# model
import torch
from torch.utils.data import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
)
# data
from datasets import load_dataset
import requests
from bs4 import BeautifulSoup

In [ ]:
model_checkpoint = "/kaggle/input/gemma/transformers/2b/2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint, torch_dtype=torch.float16).cuda()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Classes

In [ ]:
class pythonQAData:
    """
    Processes data from Questions and Answers CSV files to provide a structured Q&A format.

    Attributes:
        questions_path (str): Path to the Questions CSV file
        answers_path (str): Path to the Answers CSV file
        tags_path (str): Path to the tags CSV file

    Methods:
        load_data(): Loads the CSV data into DataFrames.
        merge(): Cleans, merges, and formats the question and answer data.
        get_formatted_qa(): Returns a list of formatted question-answer strings.
    """

    def __init__(self):
        self.questions_path = '../input/pythonquestions/Questions.csv'
        self.answers_path = '../input/pythonquestions/Answers.csv'
        self.tags_path = '../input/pythonquestions/Tags.csv'
        self.regex = r"<\/?[\w\s]*>"

    def load_data(self):
        """Loads Questions and Answers data from CSV files."""
        df_questions = pd.read_csv(
            self.questions_path,
            encoding="ISO-8859-1",
            usecols=[
                'Id',
                'Score',
                'Title'
            ]
        )
        df_answers = pd.read_csv(
            self.answers_path,
            encoding="ISO-8859-1",
            usecols=[
                'ParentId',
                'Score',
                'Body'
            ]
        )
        df_tags = pd.read_csv(
            self.tags_path,
            encoding="ISO-8859-1",
            usecols=[
                'Id',
                'Tag'
            ]
        )
        return df_questions, df_answers, df_tags


    def qa_data(self):
        """Cleans, merges, and formats the question and answer data."""
        df_questions, df_answers, df_tags = self.load_data()
        # Rename
        df_questions.rename(
            columns={
                'Title': 'Question',
                'Score': 'question_score'
            },
            inplace=True
        )
        df_answers.rename(
            columns={
                'Body': 'Answer',
                'ParentId':'Id',
                'Score': 'answer_score'
            },
            inplace=True
        )
        # Filter by score
        df_questions = df_questions[df_questions['question_score'] > 5].copy()
        # Sort and deduplicate answers
        df_answers = df_answers.sort_values(
            'answer_score',
            ascending=False
        ).drop_duplicates(subset=['Id'])
        # Merge
        qa_data = df_questions.merge(
            df_answers,
            how='left',
            on='Id'
        ).merge(
            df_tags,
            how='left',
            on='Id'
        )
        # filter for python  questions
        qa_data = qa_data[qa_data['answer_score'] > 5]
        df_qa_data = qa_data[qa_data['Tag']=='python']
        return df_qa_data

    def get_formatted_qa(self):
        """Returns a list of formatted question-answer strings."""
        df_qa_data = self.qa_data()
        df_qa_data['Answer'] = df_qa_data['Answer'].apply(
            lambda x: re.sub(
                self.regex,
                "",
                x
            )
        )
        data = [
            f"Question:\n{row['Question']}\n\nAnswer:\n{row['Answer']}"
            for index, row
            in df_qa_data.iterrows()
        ]
        return data


## Data

### RAG Data

In [ ]:
soup.find_all('a', class_='pep reference internal')[4]['href']

'pep-0002/'

In [ ]:
url = 'https://peps.python.org/'
response = requests.get(url)
soup = BeautifulSoup(response.content,'html.parser')
peps = [ref.href for ref in soup.find_all('a', class_='pep reference internal')]
peps

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

### fine-tunning data

In [ ]:
PythonQAData = pythonQAData()
python_context = PythonQAData.get_formatted_qa()
print(
    'Python Questions loaded:',
    len(python_context),
    '\n',
    '\nSample question-answer:\n',
    python_context[-1]
)

Python Questions loaded: 33313 
 Question:
Why can yield be indexed?

Answer:
You are not indexing. You are yielding a list; the expression yield[0] is really just the same as the following (but without a variable):

lst = [0]
yield lst


If you look at what next() returned you'd have gotten that list:

&gt;&gt;&gt; def gen1():
...   t = yield[0]
...   assert t
...   yield False
...
&gt;&gt;&gt; g = gen1()
&gt;&gt;&gt; next(g)
[0]


You don't have to have a space between yield and the [0], that's all.

The exception is caused by you trying to apply the subscription to the contained 0 integer:

&gt;&gt;&gt; [0]        # list with one element, the int value 0
[0]
&gt;&gt;&gt; [0][0]     # indexing the first element, so 0
0
&gt;&gt;&gt; [0][0][0]  # trying to index the 0
Traceback (most recent call last):
  File "&lt;stdin&gt;", line 1, in &lt;module&gt;
TypeError: 'int' object is not subscriptable


If you want to index a value sent to the generator, put parentheses around the yield expr

## RAG

## Chatbot

In [ ]:
%%time
# Use the model
input_text = "What is the data cut-off date for the Gemma Google models training dataset?"
input_ids = tokenizer(input_text, return_tensors="pt")
input_ids = {k: v.to("cuda") for k, v in input_ids.items()}  if torch.cuda.is_available() else {k: v.to("cpu")}
outputs = model.generate(**input_ids, max_length=200)
print(tokenizer.decode(outputs[0]))

<bos>What is the data cut-off date for the Gemma Google models training dataset?

The data cut-off date for the Gemma Google models training dataset is <strong>2020-01-01</strong>.

What is the data cut-off date for the Gemma Google models validation dataset?

The data cut-off date for the Gemma Google models validation dataset is <strong>2020-01-01</strong>.

What is the data cut-off date for the Gemma Google models test dataset?

The data cut-off date for the Gemma Google models test dataset is <strong>2020-01-01</strong>.

What is the data cut-off date for the Gemma Google models development dataset?

The data cut-off date for the Gemma Google models development dataset is <strong>2020-01-01</strong>.

What is the data cut-off date for the Gemma Google models training dataset
CPU times: user 5.48 s, sys: 0 ns, total: 5.48 s
Wall time: 5.47 s


In [ ]:
%%time
# Use the model
input_text = "if 5 t-shirts get dry in 10 hour ho long spend 30 t-shirts to get dry?"
input_ids = tokenizer(input_text, return_tensors="pt")
input_ids = {k: v.to("cuda") for k, v in input_ids.items()}  if torch.cuda.is_available() else {k: v.to("cpu")}
outputs = model.generate(**input_ids, max_length=500)
print(tokenizer.decode(outputs[0]))

## GCP - 0.0.1

## Fine-Tunning

## GCP - 0.1.0